In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)
library(data.table)
library(dplyr)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/cellranger_arc/refdata-cellranger-arc-GRCh38-2020-A-2.0.0/genes/genes.gtf.gz"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/WTC11/1.Genome_wide_prediction/SCENT/SCENT_2500.250325/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/WTC11_cluster5/WTC11_cluster5/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/WTC11_HPC_lenient/cluster_5/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/WTC11/1.Genome_wide_prediction/SCENT/SCENT_25000.250328/"
celltype = "WTC11_cluster5"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

Prepare SCENT meta data

In [9]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = celltype)
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AAACAGCCAAACCCTA-1,AAACAGCCAAACCCTA-1,18887,1818,WTC11_cluster5,9.846229,0.09625668
AAACCAACAACATAAG-1,AAACCAACAACATAAG-1,11769,1591,WTC11_cluster5,9.373224,0.13518566
AAACCAACATGAGCAG-1,AAACCAACATGAGCAG-1,19356,1871,WTC11_cluster5,9.870758,0.09666253
AAACCGAAGATTCCTT-1,AAACCGAAGATTCCTT-1,36970,4597,WTC11_cluster5,10.517862,0.12434406
AAACCGCGTACTTCAC-1,AAACCGCGTACTTCAC-1,19591,3910,WTC11_cluster5,9.882826,0.19958144
AAACCGGCAATCGCAC-1,AAACCGGCAATCGCAC-1,19538,1961,WTC11_cluster5,9.880117,0.10036851
AAACCGGCAGCTAACC-1,AAACCGGCAGCTAACC-1,23957,1650,WTC11_cluster5,10.084016,0.06887340
AAACGCGCAGGATGGC-1,AAACGCGCAGGATGGC-1,2466,107,WTC11_cluster5,7.810353,0.04339011
AAACGCGCATATAACC-1,AAACGCGCATATAACC-1,27471,2325,WTC11_cluster5,10.220886,0.08463471


RUN SCENT prediction

In [10]:
RhpcBLASctl::blas_set_num_threads(1)
Sys.setenv(OPENBLAS_NUM_THREADS=1)
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
                bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [11]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

Warning message in dir.create(paste(dir.output, "chr", sep = "/"), recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/WTC11/1.Genome_wide_prediction/SCENT/SCENT_25000.250328//chr' already exists”


In [12]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

In [13]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.01]
  pairs.E2G.chr.keep = pairs.E2G.chr[!pairs.E2G.chr$PairName %in% pairs.E2G.chr.res$PairName]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna.rename[rownames(matrix.rna.rename) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [14]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [15]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 1200583 ranges and 11 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
  chr1.chr1     chr1 100034573-100034728      * |        RTCA
  chr1.chr1     chr1 100233547-100234004      * |    RTCA-AS1
  chr1.chr1     chr1 100233547-100234004      * |     SLC30A7
  chr1.chr1     chr1 100421517-100422500      * |    RTCA-AS1
  chr1.chr1     chr1   10046013-10047382      * |      GPR157
        ...      ...                 ...    ... .         ...
  chrX.chrX     chrX     9995697-9996195      * |       CLCN4
  chrX.chrX     chrX     9995697-9996195      * |      GPR143
  chrX.chrX     chrX     9995697-9996195      * |     SHROOM2
  chrX.chrX     chrX     9995697-9996195      * |       TBL1X
  chrX.chrX     chrX     9995697-9996195      * |        WWC3
                          PeakName               PairName       beta
                       <character>            <character>  <numer

In [16]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
df.output[,"FDR"] = p.adjust(df.output$boot_basic_p, method = "BH")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed,FDR
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100034573,100034728,RTCA,WTC11_cluster5,-0.6430167,2.334365e-01,-2.75456863,5.876956e-03,0.00824,2.084073,-1,-2.084073,0.70311329
chr1,100233547,100234004,RTCA-AS1,WTC11_cluster5,0.9731452,4.115802e-01,2.36441176,1.805873e-02,0.01224,1.912219,1,1.912219,0.75234811
chr1,100233547,100234004,SLC30A7,WTC11_cluster5,0.2956173,9.964607e-02,2.96667319,3.010408e-03,0.00312,2.505845,1,2.505845,0.48926580
chr1,100421517,100422500,RTCA-AS1,WTC11_cluster5,-15.4069280,8.467772e+02,-0.01819478,9.854835e-01,0.00008,4.096910,-1,-4.096910,0.04678356
chr1,10046013,10047382,GPR157,WTC11_cluster5,-15.4928098,7.994995e+02,-0.01937813,9.845395e-01,0.00008,4.096910,-1,-4.096910,0.04678356
chr1,1005263,1005713,CCNL2,WTC11_cluster5,0.4029806,1.262056e-01,3.19304990,1.407786e-03,0.00472,2.326058,1,2.326058,0.57512958
chr1,100575095,100575942,EXTL2,WTC11_cluster5,0.3375832,1.092625e-01,3.08965238,2.003909e-03,0.00384,2.415669,1,2.415669,0.53076660
chr1,10064736,10065750,MIR34AHG,WTC11_cluster5,0.8511815,2.268238e-01,3.75261140,1.750020e-04,0.00024,3.619789,1,3.619789,0.11366466
chr1,100719355,100719975,RTCA,WTC11_cluster5,0.3909779,1.358495e-01,2.87802151,4.001778e-03,0.00872,2.059484,1,2.059484,0.72290317


In [17]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4          data.table_1.15.2    Matrix_1.6-5        
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges